# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_foods -c establishments --drop --jsonArray establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'epa', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_foods']


In [4]:
# assign the uk_food database to a variable name
food_db = mongo['uk_foods']

In [5]:
# review the collections in our new database
print(food_db.list_collection_names())

['establishments']


In [6]:
# review a document in the establishments collection
pprint(food_db.establishments.find_one())

{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Tea Room',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 551803,
 'LocalAuthorityBusinessID': 'PI/000070948',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64a9c703c5b9b2cd98ecfc4e'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/551803',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate':

In [7]:
# assign the collection to a variable
establishments = food_db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [10]:
# Create a dictionary for the new restaurant data
penang_data = {
        "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [12]:
# Insert the new restaurant into the collection

result = establishments.insert_one(penang_data)


In [13]:
# Check that the new restaurant was inserted
if result.inserted_id:
    print('result inserted successfully')
else:
    print('restaurant not found')

result inserted successfully


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [17]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType' : 'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID': 1, 'BusinessType': 1}

query_results = establishments.find(query, fields)
pprint(query_results[0])

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('64a9c703c5b9b2cd98ecfc4e')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [21]:
# Update the new restaurant with the correct BusinessTypeID
filter_query = {"BusinessName":"Penang Flavours"}
update_query = {'$set' : {'BusinessTypeID' : 1}}
business_id_update = establishments.update_one(filter_query, update_query)

In [22]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one(filter_query)
pprint(updated_restaurant)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64ac88f889e01e733157594a'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [23]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_query = {'LocalAuthorityName' : 'Dover'}
print("Number of documents for Dover: ", establishments.count_documents(dover_query))

Number of documents for Dover:  994


In [24]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_dover = establishments.delete_many(dover_query)

In [25]:
# Check if any remaining documents include Dover
print("Number of documents for Dover: ", establishments.count_documents(dover_query))

Number of documents for Dover:  0


In [26]:
# Check that other documents remain with 'find_one'
pprint(food_db.establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64a9c703c5b9b2cd98ecff34'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [30]:
# Change the data type from String to Decimal for longitude and latitude

establishments.update_many({}, [
    {
        '$set': {
            'geocode.latitude': {'$toDouble': '$geocode.latitude'},
            'geocode.longitude': {'$toDouble': '$geocode.longitude'}
        }
    }
])

Use `update_many` to convert `RatingValue` to integer numbers.

In [31]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [33]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [
    {
        '$set': {
            'RatingValue': {'$toInt': '$RatingValue'}
        }
    }
])

In [38]:
# Check that the coordinates and rating value are now numbers
type_example = establishments.find_one()

latitude_type = type(type_example['geocode']['latitude'])
longitude_type = type(type_example['geocode']['longitude'])
rating_value_type = type(type_example['RatingValue'])

print(latitude_type)
print(longitude_type)
print(rating_value_type)



<class 'float'>
<class 'float'>
<class 'int'>
